In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def pi_digits(x):
    """Generate x digits of Pi."""
    k,a,b,a1,b1 = 2,4,1,12,4
    while x > 0:
        p,q,k = k * k, 2 * k + 1, k + 1
        a,b,a1,b1 = a1, b1, p*a + q*a1, p*b + q*b1
        d,d1 = a/b, a1/b1
        while d == d1 and x > 0:
            yield int(d)
            x -= 1
            a,a1 = 10*(a % b), 10*(a1 % b1)
            d,d1 = a/b, a1/b1

digits = [int(n) for n in list(pi_digits(50))]

In [3]:
e = '27182818284590452353602874713526624977572470936999595749669676277240766303535475945713821785251664274274663919320030599218174135966290435729003342952605956307381323286279434907632338298807531952510190115738341879307021540891499348841675092447614606680822648001684774118537423454424371075390777449920695517027618386062613313845830007520449338265602976067371132007093287091274437470472306969772093101416928368190255151086574637721112523897844250569536967707854499699679468644549059879316368892300987931277361782154249992295763514822082698951936680331825288693984964651058209392398294887933203625094431173012381970684161403970198376793206832823764648042953118023287825098194558153017567173613320698112509961818815930416903515988885193458072738667385894228792284998920868058257492796104841984443634632449684875602336248270419786232090021609902353043699418491463140934317381436405462531520961836908887070167683964243781405927145635490613031072085103837505101157477041718986106873969655212671546889570350354'

In [4]:
pi = '3,1415926535 8979323846 2643383279 5028841971 6939937510 5820974944 5923078164 0628620899 8628034825 3421170679 8214808651 3282306647 0938446095 5058223172 5359408128 4811174502 8410270193 8521105559 6446229489 5493038196 4428810975 6659334461 2847564823 3786783165 2712019091 4564856692 3460348610 4543266482 1339360726 0249141273 7245870066 0631558817 4881520920 9628292540 9171536436 7892590360 0113305305 4882046652 1384146951 9415116094 3305727036 5759591953 0921861173 8193261179 3105118548 0744623799 6274956735 1885752724 8912279381 8301194912 9833673362 4406566430 8602139494 6395224737 1907021798 6094370277 0539217176 2931767523 8467481846 7669405132 0005681271 4526356082 7785771342 7577896091 7363717872 1468440901 2249534301 4654958537 1050792279 6892589235 4201995611 2129021960 8640344181 5981362977 4771309960 5187072113 4999999837 2978049951 0597317328 1609631859 5024459455 3469083026 4252230825 3344685035 2619311881 7101000313 7838752886 5875332083 8142061717 7669147303 5982534904 2875546873 1159562863 8823537875 937519577818577805321712268066130019278766111959092164201989'

In [5]:
pi = pi.replace(' ', '').replace(',', '')

In [6]:
pi_list = list()
for i in range(len(pi)):
    pi_list.append(pi[i])
display(len(pi_list))
e_list = list()
for i in range(len(e)):
    e_list.append(e[i])
display(len(e_list))

1001

1001

In [7]:
pi_list = list(map(int, pi_list))
e_list = list(map(int, e_list))

In [8]:
def calc_digits(signs, pi, e):
    obs = np.zeros((2, 10))
    obs[0][pi[0]] += 1
    obs[1][e[0]] += 1
    
    for i in range(1, signs+1):
        obs[0][pi[i]] += 1
        obs[1][e[i]] += 1
    return obs

In [9]:
calc_digits(400, pi_list, e_list)

array([[39., 43., 44., 40., 47., 39., 42., 24., 44., 39.],
       [44., 35., 43., 45., 43., 33., 39., 50., 29., 40.]])

In [10]:
for signs in {400, 700, 1000}:
    print('Signs {}:'.format(signs))
    print('chi2_stat: {}, pvalue: {}'.format(stats.chi2_contingency(calc_digits(signs, pi_list, e_list))[0],
          stats.chi2_contingency(calc_digits(signs, pi_list, e_list))[1]))

Signs 400:
chi2_stat: 14.44620357241636, pvalue: 0.10731468173609264
Signs 1000:
chi2_stat: 3.8767160583485762, pvalue: 0.9193376564361481
Signs 700:
chi2_stat: 3.5268665117934015, pvalue: 0.9397120235979187


In [11]:
def permutation_test(N, pi, e, signs):
    
    our_stat = stats.chi2_contingency(calc_digits(signs, pi, e))[0] 
    
    sample = pi[0:signs+1] + e[0:signs+1]
    per_stats = list()
    per_stats.append(our_stat)
    
    for i in range(N):
        np.random.shuffle(sample)
        per_stats.append(stats.chi2_contingency(calc_digits(signs, sample[0:signs+1], sample[signs+1:]))[0])
        
    per_stats.sort()
    
    pos = per_stats.index(our_stat, 0, len(per_stats))
    
    return (len(per_stats) - pos + 1)/len(per_stats)

In [29]:
permutation_test(1000, pi_list, e_list, 400)

0.1048951048951049

In [30]:
permutation_test(1000, pi_list, e_list, 700)

0.9430569430569431

In [31]:
permutation_test(1000, pi_list, e_list, 1000)

0.9440559440559441